### Data source 

https://www.kaggle.com/jboysen/global-food-prices

In [1]:
import numpy as np
import pandas as pd
import string 
import numpy as np
import pandas as pd
import time
from datetime import date
from calendar import monthrange
import xlsxwriter
import os

In [2]:
df = pd.read_csv(os.path.join(os.getcwd(),"wfp_food_prices_database.csv"))

/Users/chou-chia-yu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
## Find all Category names realted to Maize 

col_lst = []

for el in list(df.cm_name.unique()):
    if "Maize meal" in el:
        pass
    elif "Maize flour" in el:
        pass
    elif "Maize" in el:
        col_lst.append(el)
    else:
        pass

col_lst    

['Maize (yellow) - Wholesale',
 'Maize - Wholesale',
 'Maize (white) - Retail',
 'Maize - Retail',
 'Maize (local) - Retail',
 'Maize (yellow) - Retail',
 'Maize (imported) - Retail',
 'Maize (local) - Wholesale',
 'Maize (white) - Wholesale',
 'Maize (white, dry) - Wholesale',
 'Maize (food aid) - Retail',
 'Maize (yellow, biofortified) - Retail']

In [5]:
df_m = df[(df.cm_name.isin(col_lst)) & (df.cur_name == 'USD')]
# df_m.um_name.unique()
df_m.head().T

,340709,340710,340711,340712,340713
adm0_id,61.0,61.0,61.0,61.0,61.0
adm0_name,Costa Rica,Costa Rica,Costa Rica,Costa Rica,Costa Rica
adm1_id,0,0,0,0,0
adm1_name,NaN,NaN,NaN,NaN,NaN
mkt_id,300,300,300,300,300
mkt_name,National Average,National Average,National Average,National Average,National Average
cm_id,67,67,67,67,67
cm_name,Maize (white) - Wholesale,Maize (white) - Wholesale,Maize (white) - Wholesale,Maize (white) - Wholesale,Maize (white) - Wholesale
cur_id,0.0,0.0,0.0,0.0,0.0
cur_name,USD,USD,USD,USD,USD


In [ ]:
# ## Use dictionary to control unit lookup table

# to_kg_dic = {
#     'libra':'0.3289',
#     'Pound':'0.454',
# }


In [6]:
## Uniform unit standard into KG

print(df_m[df_m['pt_name']=='Retail'].um_name.unique())
print(df_m[df_m['pt_name']=='Wholesale'].um_name.unique())
# df_m.um_name.unique()
# df_m['pt_name'].unique()



['Libra' 'Pound' 'KG']
['MT' 'KG' '100 Pounds' '45 KG' '46 KG']


In [7]:
tb = pd.pivot_table(df_m,
    index=['adm0_name','cm_name','cur_name','pt_name','mp_month','mp_year','um_name'],
    values=['mp_price'],
    aggfunc={
        'mp_price':np.mean
    }
).reset_index()

tb

,adm0_name,cm_name,cur_name,pt_name,mp_month,mp_year,um_name,mp_price
0,Costa Rica,Maize (white) - Wholesale,USD,Wholesale,1,2008,MT,358.495000
1,Costa Rica,Maize (white) - Wholesale,USD,Wholesale,1,2009,MT,476.250000
2,Costa Rica,Maize (white) - Wholesale,USD,Wholesale,1,2010,MT,528.390000
3,Costa Rica,Maize (white) - Wholesale,USD,Wholesale,1,2011,MT,723.300000
4,Costa Rica,Maize (white) - Wholesale,USD,Wholesale,1,2012,MT,753.647500
...,...,...,...,...,...,...,...,...
1146,Zimbabwe,Maize - Retail,USD,Retail,12,2014,KG,0.302500
1147,Zimbabwe,Maize - Retail,USD,Retail,12,2015,KG,0.430000
1148,Zimbabwe,Maize - Retail,USD,Retail,12,2016,KG,0.395556
1149,Zimbabwe,Maize - Retail,USD,Retail,12,2017,KG,0.320909


In [50]:
# tb.rename(columns={"adm0_name":"Country"})

array(['Costa Rica', 'Ecuador', 'El Salvador', 'Honduras', 'Nicaragua',
       'Panama', 'Timor-Leste', 'Zimbabwe'], dtype=object)

In [8]:
## Generate excel writer
filename ='Maize_price_in_usd_country_{}.xlsx'.format(date.today().strftime("%Y%m%d"))
path = os.path.join(os.getcwd(),'Results')
os.makedirs(path,exist_ok=True) #Create directory if not exit

dirpath = os.path.join(path,filename)


writer = pd.ExcelWriter(dirpath, engine = 'xlsxwriter')
workbook  = writer.book

In [9]:
# set provider page format 
def set_form(sheet,data):

    format1 = workbook.add_format({'font_color': '#FF0000','bold': True,                                     
                                         'valign': 'fill',
                                         'align': 'justify',
                                         'text_wrap': True})
    money = workbook.add_format({'num_format': '#,##0.00'})
    
    sheet.freeze_panes(1, 3)
    header = data.columns
    row = data.shape[0]+1
    alph_up =list(string.ascii_letters[26:])
    (max_row, max_col) = data.shape
    column_settings = [{'header': column} for column in header]
    sheet.add_table(0, 0, max_row, max_col - 1, {'style': 'Table Style Medium {}'.format("1"),'columns': column_settings})

    for col_num,alph in enumerate(zip(header,alph_up)):
        order = '{}2:{}{}'.format(alph[1],alph[1],row)
        if ("mp_price" in alph[0]):
            sheet.set_column(order, 12,money)
        else:
            sheet.set_column(order, 12)

array(['Costa Rica', 'Ecuador', 'El Salvador', 'Honduras', 'Nicaragua',
       'Panama', 'Timor-Leste', 'Zimbabwe'], dtype=object)

In [11]:
for el in tb.adm0_name.unique():
    df_res = tb[tb["adm0_name"]==el]
#     display(df_res)
    df_res.to_excel(writer, sheet_name=el,index=False)
    sheet = writer.sheets['{}'.format(el)]
    set_form(sheet,df_res)
    
    

In [12]:
writer.save()